In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

"""Change to the data folder"""
new_path = "../new_train/"
val_path = "../new_val_in/"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, val=False, transform=True,scalers=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
#         if transform:
#             self.hasFit = False
#             self.scaler = [MinMaxScaler() for i in range(6)]
#             self.columns = ['p_in','v_in','p_out','v_out','lane','lane_norm']
#         if scalers:
#             self.scaler = scalers
#             self.hasFit = True
        self.val = val
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        


        return data


train_dataset  = ArgoverseDataset(data_path=new_path)
val_dataset = ArgoverseDataset(data_path=val_path,val=True)


### Create a loader to enable batch processing

In [79]:
batch_sz = 1

def closest_node(node, nodes):
    deltas = nodes - node
    dist_2 = np.einsum('ij,ij->i', deltas, deltas)
    return np.argmin(dist_2)

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inpTotal = []
    out = []
    city = []
#     print("pIn",batch[0]['p_in'])
    numbRows = 60
    for scene in batch:
        """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """

        agentIndex = numpy.where(scene['track_id'] == scene['agent_id'])[0][0]
        
    
        return [scene['p_in'][agentIndex],scene['v_in'][agentIndex],agentIndex,scene['p_out'][agentIndex],scene['v_out'][agentIndex]]

train_loader = DataLoader(train_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [28]:
batch_sz = 1
def val_collate(batch):
    agentIds = []
    trackIds = []
    sceneIdxs = []
    inpTotal = []
    city = []
    numbRows = 60
    for scene in batch:
        """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
        agentIds.append(scene['agent_id'])
        trackIds.append(scene['track_id'])
        sceneIdxs.append(scene['scene_idx'])

        agentIndex = numpy.where(scene['track_id'] == scene['agent_id'])[0][0]
       

        return [scene['p_in'][agentIndex],scene['v_in'][agentIndex],sceneIdxs,agentIds,trackIds]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=val_collate, num_workers=0)

### Training


In [100]:
from statistics import mean
import random
import numpy as np
n_epochs = 30

loss_avg = []
rangeArray = []
for x in range(20):
    rangeArray.append(0.25 + (x * 0.001))
    
for i in range(len(rangeArray)):
    loss_array = []
    loss_avg.append(loss_array)

for i_epoch in range(n_epochs):
    for i_batch, sample_batch in enumerate(train_loader):
        inp_pos,inp_velocity, agentIndex, out_pos,out_velocity= sample_batch
    #     print("inp_velocity",inp_velocity)
    #     print(inp_velocity[:,0])
        for i in range(len(rangeArray)):
            x_avg_velocity = inp_velocity[0,0]
            y_avg_velocity = inp_velocity[0,1]  
#                 print(rangeArray[i])
            for x in range(1,19):
                x_avg_velocity = (x_avg_velocity * (1-rangeArray[i])) + (inp_velocity[x,0] * rangeArray[i])
                y_avg_velocity = (y_avg_velocity * (1-rangeArray[i])) + (inp_velocity[x,1] * rangeArray[i])
        #     print("last position ",inp_pos)
            position = numpy.zeros((30,2))
            position[0][0] =  x_avg_velocity * 0.1 + inp_pos[-1,0]
            position[0][1] =  x_avg_velocity * 0.1 + inp_pos[-1,1]
            for x in range(1,30):
                position[x][0] = x_avg_velocity * 0.1 + position[x-1][0]
                position[x][1] = y_avg_velocity * 0.1 + position[x-1][1]
            position = torch.FloatTensor(position)
            out_pos = torch.FloatTensor(out_pos)
#             print(out)
            loss = torch.sqrt((torch.mean((position-out_pos)**2)))

            loss_avg[i].append(loss.item())
#                 print(rangeArray[i])
            if i_batch % 10000 == 0:
                print("increment " ,rangeArray[i] , " loss ",mean(loss_avg[i]))
                loss_avg[i] = []



start
increment  0.25  loss  1.5576459169387817
increment  0.251  loss  1.5592690706253052
increment  0.252  loss  1.560890793800354
increment  0.253  loss  1.5625195503234863
increment  0.254  loss  1.564119815826416
increment  0.255  loss  1.565721035003662
increment  0.256  loss  1.567312240600586
increment  0.257  loss  1.5688890218734741
increment  0.258  loss  1.5704461336135864
increment  0.259  loss  1.5720059871673584
increment  0.26  loss  1.573554277420044
increment  0.261  loss  1.5750901699066162
increment  0.262  loss  1.5766161680221558
increment  0.263  loss  1.5781238079071045
increment  0.264  loss  1.579621434211731
increment  0.265  loss  1.5811105966567993
increment  0.266  loss  1.5825883150100708
increment  0.267  loss  1.5840578079223633
increment  0.268  loss  1.5855042934417725
increment  0.269  loss  1.5869425535202026
increment  0.25  loss  2.36631618277356
increment  0.251  loss  2.3662976601734758
increment  0.252  loss  2.3662916932195426
increment  0.253

NameError: name 'model' is not defined

In [101]:
import numpy as np
from statistics import mean
# import pandas as pd

save_file = "Physics.csv"


header = ["ID"]
header += ["v"+str(x) for x in range(1, 61)]
print(header)

with open(save_file, 'w') as f:
    f.write(",".join(header)+"\n")


#         #Changed to 4 because we don't need lane and lane norm
batch_sz = 1
     
for i_batch, sample_batch in enumerate(val_loader):
    if i_batch % 100 == 0:
        print("batch #: ", i_batch * batch_sz)

    inp_pos,inp_velocity, scene_idx, agent_ids, track_ids= sample_batch
#     print("inp_velocity",inp_velocity)
#     print(inp_velocity[:,0])
    x_avg_velocity = inp_velocity[0,0]
    y_avg_velocity = inp_velocity[0,1]    
    for x in range(1,19):
        x_avg_velocity = (x_avg_velocity * 0.745) + (inp_velocity[x,0] * 0.255)
        y_avg_velocity = (y_avg_velocity * 0.745) + (inp_velocity[x,1] * 0.255)
#     print("last position ",inp_pos)
    position = numpy.zeros((30,2))
    position[0][0] =  x_avg_velocity * 0.1 + inp_pos[-1,0]
    position[0][1] =  x_avg_velocity * 0.1 + inp_pos[-1,1]
    for x in range(1,30):
        position[x][0] = x_avg_velocity * 0.1 + position[x-1][0]
        position[x][1] = y_avg_velocity * 0.1 + position[x-1][1]
#     print("position",position)


                           
 

    with open(save_file, "a") as f:
        row = ""
        for x in range(len(position) - 1):
            row += str(position[x][0]) + "," + str(position[x][1]) + ","
        row += str(position[-1][0]) + "," + str(position[-1][1])
#             print(scene_idx)
        f.write(str(scene_idx[i_scene])+"," + row +"\n")



['ID', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 'v58', 'v59', 'v60']
batch #:  0
batch #:  100
batch #:  200
batch #:  300
batch #:  400
batch #:  500
batch #:  600
batch #:  700
batch #:  800
batch #:  900
batch #:  1000
batch #:  1100
batch #:  1200
batch #:  1300
batch #:  1400
batch #:  1500
batch #:  1600
batch #:  1700
batch #:  1800
batch #:  1900
batch #:  2000
batch #:  2100
batch #:  2200
batch #:  2300
batch #:  2400
batch #:  2500
batch #:  2600
batch #:  2700
batch #:  2800
batch #:  2900
batch #:  3000
batch #:  3100
